In [1]:
import matplotlib.pyplot as plt
%matplotlib inline

import os
import argparse
import torch
import torch.nn as nn
from torchvision import datasets, models, transforms

from advertorch.utils import predict_from_logits
from advertorch_examples.utils import get_mnist_test_loader
from advertorch_examples.utils import _imshow

from tqdm import tqdm
from time import sleep

import time

from sklearn.preprocessing import normalize
import numpy as np
import matplotlib.pyplot as plt

torch.manual_seed(0)
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [25]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

###########################
from torchvision import datasets, transforms

test_transforms = transforms.Compose([transforms.Resize((224,224)),
                                      transforms.ToTensor(),
                                      transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])])

test_transforms= transforms.Compose([
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

test_data = datasets.ImageFolder('data/data/test/', test_transforms)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=1,shuffle=True)
test_class_names = test_data.classes

print (test_data.classes)

['go', 'goForward', 'goLeft', 'stop', 'stopLeft', 'warning', 'warningLeft']


In [26]:
filename = "models/resnet_model_acc_95.pt"
use_cuda=True
device = torch.device('cuda')

In [27]:
model_ft = models.resnet18(pretrained=True).to(device)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, len(class_names)).to(device)

# Load the pretrained model
model_ft.load_state_dict(torch.load(filename, map_location='cpu'))
# Set the model in evaluation mode. In this case this is for the Dropout layers
model_ft.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

In [62]:
model_ft_ext = models.resnet18(pretrained=True).to(device)

fearures = []
labels = []
for cln_data, true_label in test_loader:
    print (true_label)
   
    cln_data = cln_data.to(device)
    
    print (cln_data.size())
    features_of_x = model_ft_ext(cln_data)
    #print (features_of_x)
    fearures.append(features_of_x)
    
    if(len(fearures)>3): break

tensor([0])
torch.Size([1, 3, 224, 224])
tensor([3])
torch.Size([1, 3, 224, 224])
tensor([0])
torch.Size([1, 3, 224, 224])
tensor([3])
torch.Size([1, 3, 224, 224])


In [55]:
#fearures = fearures.detach().numpy() 

In [57]:
dist = np.linalg.norm(fearures[0].cpu().detach().numpy() - fearures[1].cpu().detach().numpy() )

In [58]:
print (dist)

1.3145702


In [59]:
dist = np.linalg.norm(fearures[1].cpu().detach().numpy() - fearures[2].cpu().detach().numpy() )

In [60]:
print (dist)

2.9783304


In [61]:
dist = np.linalg.norm(fearures[0].cpu().detach().numpy() - fearures[2].cpu().detach().numpy() )
print (dist)

3.0207014
